In [1]:
from _shared import *
from tqdm.notebook import tqdm
from hashlib import md5
from copy import deepcopy

## Задача 1. Функция распределения экспериментов по бакетам

In [2]:
from copy import deepcopy

def add_experiment(experiment, buckets):
    """
    Проверяет можно ли добавить эксперимент, добавляет если можно.

    Распределять эксперименты нужно так, чтобы умещалось как можно больше экспериментов.

    :param experiment (dict): параметры эксперимента, который нужно запустить.
        Ключи словаря:
            - id - идентификатор эксперимента.
            - buckets_count - необходимое количество бакетов.
            - conflicts - список идентификаторов экспериментов, которые нельзя проводить
                одновременно на одних и тех же пользователях.
    :param buckets (list[list[int]]): список бакетов, в каждом бакете перечислены
            идентификаторы экспериментов, которые в нём проводятся.

    :return (success, buckets):
        success (boolean) - можно ли добавить эксперимент, True - можно, иначе - False
        buckets (list[list[int]]) - обновлённый список бакетов с добавленным экспериментом,
            если эксперимент добавить можно.
    """
    buckets_count = experiment['buckets_count']
    
    if buckets_count > len(buckets):
        return (False, buckets)
    
    if not experiment['conflicts']:
        conflicts = ['_', ]
    else:
        conflicts = experiment['conflicts']
        
    buckets_new = deepcopy(buckets)
    c_set = set(conflicts)
    
    #for c in conflicts:
    #    for i in range(len(buckets_new)):

    for i in range(len(buckets_new)):
        if not c_set.intersection(buckets_new[i]) and (buckets_count > 0):
            buckets_new[i].append(experiment['id'])
            buckets_count -= 1

    if buckets_count != 0:
        return (False, buckets)
    
    return (True, buckets_new)

In [3]:
total_buckets_count = 4
buckets = [[] for _ in range(total_buckets_count)]

In [4]:
success, buckets = add_experiment({'id': 0, 'buckets_count': 5, 'conflicts': []}, buckets)
# для эксперимента необходимо больше бакетов, чем доступно (5 > 4)
# success, buckets = False, [[], [], [], []]
success, buckets

(False, [[], [], [], []])

In [5]:
success, buckets = add_experiment({'id': 1, 'buckets_count': 4, 'conflicts': []}, buckets)
# success, buckets = True, [[1], [1], [1], [1]]
success, buckets

(True, [[1], [1], [1], [1]])

In [6]:
success, buckets = add_experiment({'id': 2, 'buckets_count': 2, 'conflicts': [3]}, buckets)
# эксперимент с id=2 может быть в любых двух бакетах их четырёх
# success, buckets = True, [[1, 2], [1], [1, 2], [1]]
success, buckets

(True, [[1, 2], [1, 2], [1], [1]])

In [7]:
success, buckets = add_experiment({'id': 3, 'buckets_count': 2, 'conflicts': [2]}, buckets)
# можем добавить в бакеты, где не запущен экперимент с id=2
# success, buckets = True, [[1, 2], [1, 3], [1, 2], [1, 3]]
success, buckets

(True, [[1, 2], [1, 2], [1, 3], [1, 3]])

In [8]:
success, buckets = add_experiment({'id': 4, 'buckets_count': 1, 'conflicts': [1]}, buckets)
# не можем добавить, так как во всех бакетах запущен эксперимент, с которым конфликт
# success, buckets = False, [[1, 2], [1, 3], [1, 2], [1, 3]]
success, buckets

(False, [[1, 2], [1, 2], [1, 3], [1, 3]])

In [9]:
tests = [
    {
        'buckets_count': 4,
        'experiments': [
            {'id': 1, 'buckets_count': 2, 'conflicts': []},
            {'id': 2, 'buckets_count': 2, 'conflicts': []},
            {'id': 3, 'buckets_count': 2, 'conflicts': []},
            {'id': 4, 'buckets_count': 5, 'conflicts': []}
        ]
    },
    {
        'buckets_count': 4,
        'experiments': [
            {'id': 1, 'buckets_count': 2, 'conflicts': [2, 4]},
            {'id': 2, 'buckets_count': 2, 'conflicts': [1]},
            {'id': 3, 'buckets_count': 2, 'conflicts': []},
            {'id': 4, 'buckets_count': 4, 'conflicts': [1]}
        ]
    },
    {
        'buckets_count': 4,
        'experiments': [
            {'id': 1, 'buckets_count': 2, 'conflicts': [3]},
            {'id': 2, 'buckets_count': 2, 'conflicts': [3]},
            {'id': 3, 'buckets_count': 2, 'conflicts': [1, 2]}
        ]
    },
    {
        'buckets_count': 4,
        'experiments': [
            {'id': 1, 'buckets_count': 2, 'conflicts': [4, 5]},
            {'id': 2, 'buckets_count': 2, 'conflicts': [4, 5]},
            {'id': 3, 'buckets_count': 2, 'conflicts': [4, 5]},
            {'id': 4, 'buckets_count': 2, 'conflicts': [1, 2, 3]},
            {'id': 5, 'buckets_count': 2, 'conflicts': [1, 2, 3]},
            {'id': 6, 'buckets_count': 4, 'conflicts': [7]},
            {'id': 7, 'buckets_count': 1, 'conflicts': [6]}
        ]
    },
]

In [10]:
for t in tests:
    buckets = [[] for _ in range(t['buckets_count'])]
    for exp in t['experiments']:
        print('Experiment: {}\nPre-Buckets: {}'.format(exp, buckets))
        success, buckets = add_experiment(exp, buckets)
        print('Success: {}\nPost-Buckets:{}\n'.format(success, buckets))
    print('\n')

Experiment: {'id': 1, 'buckets_count': 2, 'conflicts': []}
Pre-Buckets: [[], [], [], []]
Success: True
Post-Buckets:[[1], [1], [], []]

Experiment: {'id': 2, 'buckets_count': 2, 'conflicts': []}
Pre-Buckets: [[1], [1], [], []]
Success: True
Post-Buckets:[[1, 2], [1, 2], [], []]

Experiment: {'id': 3, 'buckets_count': 2, 'conflicts': []}
Pre-Buckets: [[1, 2], [1, 2], [], []]
Success: True
Post-Buckets:[[1, 2, 3], [1, 2, 3], [], []]

Experiment: {'id': 4, 'buckets_count': 5, 'conflicts': []}
Pre-Buckets: [[1, 2, 3], [1, 2, 3], [], []]
Success: False
Post-Buckets:[[1, 2, 3], [1, 2, 3], [], []]



Experiment: {'id': 1, 'buckets_count': 2, 'conflicts': [2, 4]}
Pre-Buckets: [[], [], [], []]
Success: True
Post-Buckets:[[1], [1], [], []]

Experiment: {'id': 2, 'buckets_count': 2, 'conflicts': [1]}
Pre-Buckets: [[1], [1], [], []]
Success: True
Post-Buckets:[[1], [1], [2], [2]]

Experiment: {'id': 3, 'buckets_count': 2, 'conflicts': []}
Pre-Buckets: [[1], [1], [2], [2]]
Success: True
Post-Bucket

In [11]:
# Solution
def add_experiment(experiment, buckets):
    # список [bucket_id, количество совместных экспериментов] для бакетов без конфликтов
    available_buckets_meta = []
    for bucket_id, bucket in enumerate(buckets):
        if set(experiment['conflicts']) & set(bucket):
            continue
        available_buckets_meta.append((bucket_id, len(bucket)))
    if len(available_buckets_meta) < experiment['buckets_count']:
        # доступных бакетов недостаточно
        return False, buckets
    # в первую очередь занимаем бакеты с большим количеством экспериментов
    # чтобы оставить свободные бакеты для несовместных экспериментов
    sorted_available_buckets_meta = sorted(available_buckets_meta, key=lambda x: -x[1])
    for bucket_id, _ in sorted_available_buckets_meta[:experiment['buckets_count']]:
        buckets[bucket_id].append(experiment['id'])
    return True, buckets

## Задача 2. Функция распределения пользователей по бакетам и группам

In [12]:
import hashlib


def get_hash_modulo(value: str, salt: str, modulo: int):
    """Вычисляем остаток от деления: (hash(value + salt)) % modulo."""
    hash_value = int(hashlib.md5(str.encode(value + salt)).hexdigest(), 16)
    return hash_value % modulo


def process_user(user_id, buckets, experiments, bucket_salt):
    """
    Определяет в какие эксперименты попадает пользователь.

    Сначала нужно определить бакет пользователя.
    Затем для каждого эксперимента в этом бакете выбрать пилотную или контрольную группу.

    :param user_id (str): идентификатор пользователя
    :param buckets (list[list[int]]): список бакетов, в каждом бакете перечислены
            идентификаторы экспериментов, которые в нём проводятся.
    :param experiments (list[dict]): список словарей с информацией об экспериментах.
        Ключи словарей:
        - id (int) - идентификатор эксперимента
        - salt (str) - соль эксперимента для распределения пользователей на
            контрольную/пилотную группы.
    :param bucket_salt (str): соль для разбиения пользователей по бакетам.
        При одной соли каждый пользователь должен всегда попадать в один и тот же бакет.
        Если изменить соль, то распределение людей по бакетам должно измениться.
    :return bucket_id, experiment_groups:
        - bucket_id (int) - номер бакета (индекс элемента в buckets)
        - experiment_groups (list[tuple]) - список пар: id эксперимента, группа.
            Группы: 'A', 'B'.
        Пример: (8, [(194, 'A'), (73, 'B')])
    """
    bucket_id = 0
    experiment_groups = [(), ()]
    
    for exp in experiments:
        hm = get_hash_modulo(exp[)
    
    return (bucket_id, experiment_groups)

In [13]:
user_id = '1001'
experiments = [{'id': 0, 'salt': '0'}, {'id': 1, 'salt': '1'}]
buckets = [[0, 1], [1], []]
bucket_salt = 'a2N4'
bucket_id, experiment_groups = process_user(user_id, buckets, experiments, bucket_salt)
# В зависимости от значений bucket_salt и солей экспериментов, можно получить один из вариантов:
# bucket_id, experiment_groups = 0, [(0, 'A'), (1, 'A')]
# bucket_id, experiment_groups = 0, [(0, 'A'), (1, 'B')]
# bucket_id, experiment_groups = 0, [(0, 'B'), (1, 'A')]
# bucket_id, experiment_groups = 0, [(0, 'B'), (1, 'B')]
# bucket_id, experiment_groups = 1, [(1, 'A')]
# bucket_id, experiment_groups = 1, [(1, 'B')]
# bucket_id, experiment_groups = 2, []